In [1]:
import pandas as pd    #dataframe to work with tables
import numpy as np     #for general calculations with arrays
import pycmap          #API for querrying the data from CMAP
import datetime
import cdsapi

import plotly.express as px   #makes nice interactive plots


import cartopy.crs as ccrs         # to plot maps with different projections
import cartopy.feature as cfeature # to plot coastlines, land, borders, etc.
import matplotlib.pyplot as plt  #makes regular plots

from matplotlib.colors import LogNorm  #to make logarithmic colormap axis

import xarray as xr  #to work with data

from scipy.interpolate import griddata   #interpolate irregularly spaced data onto a grid
import scipy as sp

In [2]:
#enter your personal API key from CMAP here
key = "0e194d46-b937-4edf-a141-05ca24dd0224"                     
#call the CMAP API using your unique key
cmap_client = pycmap.API(token=key)

In [3]:
# Download the chl data
YEAR_MIN, YEAR_MAX = (1970, 2020)
LAT_MIN, LAT_MAX = (30, 48)
LON_MIN, LON_MAX = (-130, -115)
modis=cmap_client.query(
         f'''
         SELECT [time], AVG(chl) AS chl FROM tblCHL_REP 
         WHERE 
         [time] BETWEEN '{YEAR_MIN}' AND '{YEAR_MAX}' AND 
         lat BETWEEN {LAT_MIN} AND {LAT_MAX} AND 
         lon BETWEEN {LON_MIN} AND {LON_MAX}
         GROUP BY [time] 
         ORDER BY [time]
         '''
         )
# Add the chl data to a dataframe
chl_dat = pd.DataFrame(modis)[:-1]
chl_dat.time = [datetime.datetime(int(t[:4]), int(t[5:7]), 1) for t in chl_dat.time]
chl_dat = chl_dat.groupby('time').mean('time')
chl_dat = chl_dat.reset_index()
del modis # free some memory by deleting the no longer needed variables.

In [4]:
# Import ERA5 data to a dataframe
ERA5_Coarse ='./PO/ERA5/ERA5_Coarse.nc'    #loading from local path
era = xr.open_dataset(ERA5_Coarse).sel(time=slice(str(YEAR_MIN), str(YEAR_MAX-1)),
                                       latitude=slice(str(LAT_MAX), str(LAT_MIN)),
                                       longitude=slice(str(360+LON_MIN), str(360+LON_MAX)))   #xarray can open different format data, netcdf is one of them
era_avg_dat = era.mean(['longitude', 'latitude']).to_dataframe()
era_avg_dat = era_avg_dat.reset_index();

In [35]:
# Merge the two dataframes
df = pd.merge(chl_dat, era_avg_dat)

In [36]:
# Bin the data into categorical variables
df_processed = pd.DataFrame()
n_bins = 5 # seems reasonable?
keys_to_drop = ['time', 't2m_F']
bin_dict = {}
for key in df.keys():
    if key not in keys_to_drop:
        new_col, bins = pd.cut(df[key], n_bins, labels=False, retbins=True)
        bin_dict[key] = bins
        df_processed.insert(0, key, new_col)

Now what I want to do is to formulate some causal hypothesis for all the variables. As an example, maybe we have edges:
$$U \to (CHL, TP), V \to (CHL, TP)$$
so that $CHL \perp_d TP | UV$. Then we can test this constraint.

In [37]:
# Group binning of U and V
df_processed.insert(0, 'uv', [n_bins*row['u10'] + row['v10'] for idx, row in df_processed.iterrows()])
uv_stratified = df_processed.groupby('uv')
pvalues_hyp = {}
for bin, group in uv_stratified.groups.items():
    if len(group) > 1:
        stat, p = sp.stats.pearsonr(df_processed['tp'][df_processed['uv'] == bin],
                                    df_processed['chl'][df_processed['uv'] == bin])
        pvalues_hyp[bin] = p
print(min(pvalues_hyp.values()))

0.003992797594578227


/var/folders/98/6vx_1r7n22g6mqrnk3j70t5c0000gn/T/ipykernel_42714/124930367.py:7: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  stat, p = sp.stats.pearsonr(df_processed['tp'][df_processed['uv'] == bin],


This tells us that we have a statistically significant correlation between $CHL$ and $TP$, even conditioned on the wind, so we must *reject* our causal hypothesis.

But what I actually want is to incorporate the temporality somehow, and there are general tools for this, e.g.:
- https://www.statsmodels.org/stable/tsa.html
- https://www.statsmodels.org/stable/contingency_tables.html
- https://causal-learn.readthedocs.io/en/latest/
- https://fentechsolutions.github.io/CausalDiscoveryToolbox/html/index.html